<h1>Yelp API</h1>
<li>https://www.yelp.com/developers/documentation/v3
<li>log into yelp (top right hand corner of the page)
<li>Click <span style="color:blue">Create App</span> on the left hand menu bar
<li>Enter app info (leave optional stuff blank)
<li>Copy the client id and client secret to a secure place (this notebook should do the trick or use a text file!)

In [1]:
CLIENT_ID = ""
API_KEY = ""

In [2]:
with open('/Users/hardeepjohar/Documents/Courses/Fall2018/API_KEYS/yelp_fusion.txt','r') as f:
    count = 0
    for line in f:
        if count == 0:
            CLIENT_ID = line.strip()
        if count == 1:
            API_KEY = line.strip()
        count+=1


In [ ]:
print(CLIENT_ID,API_KEY)

In [4]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business

<h3>Now we can get reviews</h3>
<li>get_reviews(location,number=15) returns the reviews of "number" (default=15) restaurants in the vicinity of "location"
<li>First, we'll write a function that gets  restaurants in the vicinity of location



In [8]:
def get_restaurants(api_key,location,number=15):
    import requests
    
    #First we get the access token
    #Set up the search data dictionary
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    return businesses

In [9]:
get_restaurants(API_KEY,"Columbia University, New York, NY")

[{'alias': 'flat-top-new-york',
  'categories': [{'alias': 'newamerican', 'title': 'American (New)'},
   {'alias': 'cafes', 'title': 'Cafes'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
  'coordinates': {'latitude': 40.810041, 'longitude': -73.958693},
  'display_phone': '(646) 820-7735',
  'distance': 424.3853457755761,
  'id': 'E2mNgb479B3BCfwi2G_KdQ',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/y2YlqtpKU2tciozvngNbsg/o.jpg',
  'is_closed': False,
  'location': {'address1': '1241 Amsterdam Ave',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'country': 'US',
   'display_address': ['1241 Amsterdam Ave', 'New York, NY 10027'],
   'state': 'NY',
   'zip_code': '10027'},
  'name': 'Flat Top',
  'phone': '+16468207735',
  'price': '$$',
  'rating': 4.0,
  'review_count': 391,
  'transactions': ['restaurant_reservation'],
  'url': 'https://www.yelp.com/biz/flat-top-new-york?adjust_creative=0eXIew5caAsZxoB9drIq4g&utm_campaign=yelp_api_v3&u

In [10]:
def get_business_review(api_key,business_id):
    import json
    import requests
    business_path = BUSINESS_PATH + business_id+"/reviews"
    url = API_HOST + business_path

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }


    response = requests.request('GET', url, headers=headers).json()
   
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

In [11]:
get_business_review(API_KEY,'flat-top-new-york')

"We made a preshow reservation for Friday at 6 pm, and definitely didn't need it, though the place was hopping by 7.\n\nThey dealt admirably with our picky...A friend who goes to Columbia and lives around the block recommended Flat Top to me, and it didn't dissapoint!\n\nWe sat at the bar and the bartender was...Flat Top is a great example of why you don't go to white-owned restaurants in black neighborhoods. Overpriced, overhyped and ultimately unappetizing...."

<h4>Finally, put all this together to get review data for the set of restaurants</h4>


In [12]:
def get_reviews(location,number=15):

    restaurants = get_restaurants(API_KEY,location,number)

    if not restaurants:
        return None
    review_list = list()
    for restaurant in restaurants:
        restaurant_name = restaurant['name']
        restaurant_id = restaurant['id']
        review_text = get_business_review(API_KEY,restaurant_id)
        
        review_list.append((restaurant_name,review_text))
    return review_list
        


In [13]:
all_snippets = get_reviews("Columbia University, New York, NY")

In [14]:
all_snippets

[('Flat Top',
  "We made a preshow reservation for Friday at 6 pm, and definitely didn't need it, though the place was hopping by 7.\n\nThey dealt admirably with our picky...A friend who goes to Columbia and lives around the block recommended Flat Top to me, and it didn't dissapoint!\n\nWe sat at the bar and the bartender was...Flat Top is a great example of why you don't go to white-owned restaurants in black neighborhoods. Overpriced, overhyped and ultimately unappetizing...."),
 ('Dun Huang',
  "I came here after class with a friend and we shared four dishes (I came to taste he came to eat) we tried three Cold noodle dishes: spicy noodle, orange...Came here on a rainy night and the restaurant was busy. Many of the items on the menu looked spicy and we don't have a high tolerance with spice so we both...Food was wonderful, we ordered the traditional Lanzhou noodles, the mung bean jelly, and the lamb kebab (delicious, with just the right touch of fattiness..."),
 ("e's BAR",
  "I have

<h2>A function that analyzes emotions</h2>


In [17]:
def get_nrc_data():
    nrc = "/Users/hardeepjohar/Documents/Courses/Fall2018/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict


def emotion_analyzer(text,emotion_dict=get_nrc_data()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

In [18]:
print("%-12s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(
        "restaurant","fear","trust","negative","positive","joy","disgust","anticip",
        "sadness","surprise"))
        
for snippet in all_snippets:
    text = snippet[1]
    result = emotion_analyzer(text)
    print("%-12s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
        snippet[0][0:10],result['fear'],result['trust'],
          result['negative'],result['positive'],result['joy'],result['disgust'],
          result['anticipation'],result['sadness'],result['surprise']))


restaurant   fear	trust negative positive joy   disgust anticip sadness surprise
Flat Top     0.00	0.03	0.01	0.03	0.01	0.00	0.01	0.01	0.00
Dun Huang    0.00	0.02	0.00	0.05	0.02	0.00	0.00	0.01	0.00
e's BAR      0.00	0.02	0.01	0.05	0.04	0.00	0.02	0.00	0.01
Lolo's Sea   0.00	0.04	0.01	0.08	0.04	0.00	0.04	0.00	0.02
Thai Marke   0.00	0.06	0.00	0.07	0.06	0.00	0.02	0.00	0.00
Friedman's   0.00	0.04	0.00	0.02	0.02	0.00	0.02	0.00	0.01
Saiguette    0.01	0.01	0.00	0.03	0.01	0.00	0.00	0.00	0.00
Dig Inn      0.00	0.02	0.01	0.01	0.01	0.00	0.00	0.00	0.00
Mokja        0.00	0.04	0.01	0.04	0.03	0.00	0.03	0.00	0.03
Jin Ramen    0.00	0.01	0.00	0.02	0.02	0.00	0.01	0.00	0.00
Marlow Bis   0.00	0.02	0.00	0.05	0.04	0.00	0.01	0.00	0.01
Pisticci     0.02	0.01	0.01	0.01	0.01	0.00	0.00	0.01	0.00
Red Hot Ho   0.00	0.04	0.00	0.04	0.03	0.00	0.04	0.00	0.02
KALBI        0.00	0.10	0.01	0.10	0.09	0.00	0.05	0.00	0.04
Mekong       0.00	0.01	0.00	0.02	0.01	0.00	0.00	0.00	0.00


<h4>Let's functionalize this</h4>
<h3>For easy of analysis, we'll do the following:</h3>
<li>generalize it so that we can analyze any document type, not just restaurant reviews
<li>output a dataframe containing the results. this will make analysis of the results easier
<li>we'll decide whether or not we should print the output from the function

In [19]:
def comparative_emotion_analyzer(text_tuples,object_name="Restaurant",print_output=False):
    if print_output:
        print("%-20s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(object_name,
                                                              "fear","trust","negative","positive",
                                                              "joy","disgust","anticip", "sadness",
                                                              "surprise"))
    import pandas as pd
    df = pd.DataFrame(columns=[object_name,'Fear','Trust','Negative',
                           'Positive','Joy','Disgust','Anticipation',
                           'Sadness','Surprise'],)
    df.set_index(object_name,inplace=True)
    
    output = df    
    for text_tuple in text_tuples:
        text = text_tuple[1] 
        result = emotion_analyzer(text)
        if print_output:
            print("%-20s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
                text_tuple[1][0:20],result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']))
        df.loc[text_tuple[0]] = [result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']]
    return output
#And test it        
comparative_emotion_analyzer(all_snippets)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Restaurant,,,,,,,,,
Flat Top,0.000000,0.025974,0.012987,0.025974,0.012987,0.0,0.012987,0.012987,0.000000
Dun Huang,0.000000,0.023529,0.000000,0.047059,0.023529,0.0,0.000000,0.011765,0.000000
e's BAR,0.000000,0.024390,0.012195,0.048780,0.036585,0.0,0.024390,0.000000,0.012195
Lolo's Seafood Shack,0.000000,0.043956,0.010989,0.076923,0.043956,0.0,0.043956,0.000000,0.021978
Thai Market,0.000000,0.058824,0.000000,0.070588,0.058824,0.0,0.023529,0.000000,0.000000
Friedman's,0.000000,0.037037,0.000000,0.024691,0.024691,0.0,0.024691,0.000000,0.012346
Saiguette,0.012987,0.012987,0.000000,0.025974,0.012987,0.0,0.000000,0.000000,0.000000
Dig Inn,0.000000,0.022727,0.011364,0.011364,0.011364,0.0,0.000000,0.000000,0.000000
Mokja,0.000000,0.037500,0.012500,0.037500,0.025000,0.0,0.025000,0.000000,0.025000


In [20]:
def analyze_nearby_restaurants(address,number=15):
    snippets = get_reviews(address,number)
    return comparative_emotion_analyzer(snippets)

#And test it    
analyze_nearby_restaurants("Columbia University",15)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Restaurant,,,,,,,,,
Flat Top,0.000000,0.025974,0.012987,0.025974,0.012987,0.0,0.012987,0.012987,0.000000
Dun Huang,0.000000,0.023529,0.000000,0.047059,0.023529,0.0,0.000000,0.011765,0.000000
e's BAR,0.000000,0.024390,0.012195,0.048780,0.036585,0.0,0.024390,0.000000,0.012195
Lolo's Seafood Shack,0.000000,0.043956,0.010989,0.076923,0.043956,0.0,0.043956,0.000000,0.021978
Thai Market,0.000000,0.058824,0.000000,0.070588,0.058824,0.0,0.023529,0.000000,0.000000
Friedman's,0.000000,0.037037,0.000000,0.024691,0.024691,0.0,0.024691,0.000000,0.012346
Saiguette,0.012987,0.012987,0.000000,0.025974,0.012987,0.0,0.000000,0.000000,0.000000
Dig Inn,0.000000,0.022727,0.011364,0.011364,0.011364,0.0,0.000000,0.000000,0.000000
Mokja,0.000000,0.037500,0.012500,0.037500,0.025000,0.0,0.025000,0.000000,0.025000


In [21]:
#Test it on some other place
analyze_nearby_restaurants("221 Baker Street, London, UK",15)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Restaurant,,,,,,,,,
Ffiona's Restaurant,0.011628,0.023256,0.023256,0.058140,0.034884,0.000000,0.011628,0.023256,0.023256
The Grazing Goat,0.000000,0.000000,0.000000,0.024691,0.000000,0.000000,0.012346,0.000000,0.012346
The Mayfair Chippy,0.000000,0.058140,0.000000,0.069767,0.034884,0.011628,0.023256,0.000000,0.034884
The Victoria,0.000000,0.023810,0.000000,0.059524,0.035714,0.000000,0.000000,0.000000,0.000000
Dishoom,0.021978,0.043956,0.054945,0.043956,0.043956,0.010989,0.043956,0.010989,0.010989
Great Queen Street,0.025000,0.000000,0.025000,0.050000,0.012500,0.000000,0.012500,0.012500,0.012500
Mother Mash,0.000000,0.000000,0.000000,0.036145,0.012048,0.000000,0.000000,0.000000,0.000000
Nambu Tei,0.013158,0.039474,0.000000,0.052632,0.052632,0.000000,0.026316,0.013158,0.026316
The Pig and Butcher,0.000000,0.011364,0.000000,0.045455,0.022727,0.000000,0.011364,0.011364,0.011364
